# Convolutional Neural Networks with Data Augmentation using Keras

---


## In this tutorial I will be using Keras with TensorFlow as backend to calssify digits from the MNIST Dataset

We will start by importing Keras.

In [1]:
import numpy
from keras import backend as K
from keras.datasets import mnist
from keras.utils import np_utils

Using TensorFlow backend.


We then import the layers of the convolutional neural network.The network consists of two main components :

1. Convolutional layers : the convolutional layer is responsible for the convolutional operation in which feature maps identifies features in the images.
and is usually followed by two types of layers which are :
>*   **Dropout** : Dropout is a regulization technique where you turn off part of the network's layers randomally to increase regulization and hense decrease overfitting. We use when the training set accuracy is muuch higher than the test set accuracy.
>*   **Max Pooling** : The maximum output in a rectangular neighbourhood. It is used to make the network more flexible to slight changes and decrease the network computationl expenses by extracting the group of pixels that are highly contributing to each feature in the feature maps in the layer.
2. Dense layers : The dense layer is a fully connected layer that comes after the convolutional layers and they give us the output vector of the Network.

As a convention in Convolutional Neural Network we decrease the dimensions of the layers as we go deeper and increase the number of feature maps to make it detect more features and decrease the number of computational cost.

![alt text](https://raw.githubusercontent.com/MoghazyCoder/Machine-Learning-Tutorials/master/assets/Untitled.png)

 

In [2]:
from keras.layers import Dense, Dropout,Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D

Sequential layers are stacked such that every layer passes its output to the next layer without you specifying extra information so we import Sequential from models

In [3]:
from keras.models import Sequential
import pandas as pd

model = Sequential()

We must specify which data format convention Keras will follow using the following line of code. Keras can accept the number of channels before other dimensions or after it but here we have to specify which convention we will use. We will use channels last which is Tensorflow's convention .

In [4]:
K.set_image_data_format('channels_last')
numpy.random.seed(0)

We should call mnist.load_data() which contains the mnist Dataset of 60,000 28x28 grayscale images of the 10 digits, along with a test set of 10,000 images.  when we call mnist.load_data() it returns two tuples one for the training set containing the images and their corresponding lables and another one for the test set.

In [8]:
X = pd.read_csv('./input/02 - train.csv')
test = pd.read_csv('./input/02 - test.csv')

y = X["label"]
X.drop(["label"], inplace = True, axis = 1)

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.2 , random_state=42)

We then reshape the samples according to TensorFlow convention which we chosed previously using "K.set_image_data_format('channels_last')" samples,rows,columns,channels as we are using channels_last if you are using channels_first you will need to change the order to samples,channels,rows,column and here we have only one channel because we are using the image in grayscale not RGB. We should also make the output in the form of one vs all (aka one hot encoding) which means that we will have 10 calsses from 0 to 9 one class for each number from 0 to 9



In [10]:
X_train = X_train.reshape(X_train.shape[0], 28, 28 , 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 28, 28 , 1).astype('float32')

<h1>Data Exploration</h1>
Let's explore the data we have as this will give us a hint on the algorithm we will use if we have to choose. Exploring data is also very important because it will tell you which accuracy metric you are going to use, if the data is balanced which means all the classes have fair contribution in the dataset regarding its numbers then we can easily use accuracy, But if the data is skewed then we won't be able to use accurace as it's results will be misleading and we may use F-beta score instead.

In [11]:
import matplotlib.pyplot as plt
print("the number of training examples = %i" % X_train.shape[0])
print("the number of classes = %i" % len(numpy.unique(y_train)))
print("Dimention of images = {:d} x {:d}  ".format(X_train[1].shape[0],X_train[1].shape[1])  )

#This line will allow us to know the number of occurrences of each specific class in the data
unique, count= numpy.unique(y_train, return_counts=True)
print("The number of occuranc of each class in the dataset = %s " % dict (zip(unique, count) ), "\n" )
 
images_and_labels = list(zip(X_train,  y_train))
for index, (image, label) in enumerate(images_and_labels[:12]):
    plt.subplot(5, 4, index + 1)
    plt.axis('off')
    plt.imshow(image.squeeze(), cmap=plt.cm.gray_r, interpolation='nearest')
    plt.title('label: %i' % label )

the number of training examples = 33600
the number of classes = 10
Dimention of images = 28 x 28  
The number of occuranc of each class in the dataset = {0: 3316, 1: 3775, 2: 3331, 3: 3414, 4: 3233, 5: 3093, 6: 3352, 7: 3508, 8: 3228, 9: 3350}  



From the previous results we can see that the dataset consists of 60000 training example each is an image of dimention 28 * 28. We can see that the number of occurances of each class is almost balanced and based on that it is safe to use accuracy as our metric later.

<h1>  Model Design and Achitecture</h1>
Now lets implement the first layer of the convolutional network as shown in the schema below and i will use a simple archicture simillar to LeCun's network a .
![alt text](https://raw.githubusercontent.com/MoghazyCoder/Machine-Learning-Tutorials/master/assets/Layer.png)
For the sequential model you just stack the layers and only specify the image input dimensions in the first layer.
Our first layer will be a convolutional layer Conv2D() where we specify the number of feature maps , the input shape and the activation function which is here relu .The relu activation function is represented mathematically by max(0,X).
We then add the max pooling layer (which is the most common kind of pooling) with a kernel of dimensions 2 * 2 .


In [12]:
from keras.layers import Dropout

model.add(Conv2D(40, kernel_size=5, padding="same",input_shape=(28, 28, 1), activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

Lets add the 2nd layer but this time we increase the feature maps .

In [13]:
model.add(Conv2D(70, kernel_size=3, padding="same", activation = 'relu'))
model.add(Conv2D(500, kernel_size=3, padding="same", activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(1024, kernel_size=3, padding="valid", activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

Now we add a flatten layer that takes the output of the CNN and flattens it and passes it as an input to the Dense Layers which passes it to the output layer.
we have used number of classes = 10 because we have 10 numbers from 0 to 9 .
every dense layer contains 300 neurons except for the output layer.
We use Softmax with the output layer to output estimated probability vector for  multi-class classification .

In [14]:
from keras.layers.core import Activation

model.add(Flatten())
model.add(Dense(units=100, activation='relu'  ))
model.add(Dropout(0.1))
model.add(Dense(units=100, activation='relu'  ))
model.add(Dropout(0.1))
model.add(Dense(units=100, activation='relu'  ))
model.add(Dropout(0.3))

model.add(Dense(10))
model.add(Activation("softmax"))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

We have to compile the model and then try training it using the fit() function which fits the training data and labels , the number of epochs and the batch_size which is the number of photos per training cycle.
The last thing that we are going to do is to evaluate the model to ensure that it doesn't overfit the trainig data .Evaluating the model is done by using the weights that resulted from the training step and using it to estimate the value of the test data that the model haven't seen before to estimate how well the model will perform in the future on new data.

if you are using cross-validation split then the convention is to split the data by 60% training set , 20% validation set and 20% test set but in the era of big data this ratio may vary according to the amount of data you have.

We have used categorical_crossentropy as the cost function for that model but what does we mean by **cost function**

#### Cost function : It is a measure of the overall loss in our network after assigning values to the parameters during the forward phase so it indicates how well the parameters were chosen during the forward probagation phase.

#### Optimizer : It is the gradiant descent algorithm that is used. We use it to minimize the cost function to approach the minimum point. We are using adam optimizer which is one of the best gradient descent algorithms. You can refere to this paper to know how it works https://arxiv.org/abs/1412.6980v8

You can use other metrics to measure the performance other than accuracy as precision or recall or F1 score. the choice depends on the problem itself. Where high recall means low number of false negatives , High precision means low number of false positives and     F1 score is a trade off between them. You can refere to this article for more about precision and recall http://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html

In [15]:
y_train = np_utils.to_categorical(y_train).astype('int32')
y_test = np_utils.to_categorical(y_test)

I will use ImageDataGenerator from keras to augment the images. Augmenting the images makes the model more robust and more generalizable when using newly unseen data like the data in the test set of the competition. There are many ways to augment the images like centering the images, normalization, rotation, shifting, and flipping and i will use some of them [here](http://) .

In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

X_train = X_train.reshape(-1,28,28,1)
X_test = X_test.reshape(-1,28,28,1)

from keras.preprocessing.image import ImageDataGenerator
X_train2 = numpy.array(X_train, copy=True) 
y_train2 = numpy.array(y_train, copy=True) 

datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    )

datagen.fit(X_train)

print(type(X_train2))
print(type(X_train))

# Concatenating the old data with the augmented data
result_x  = numpy.concatenate((X_train, X_train2), axis=0)
result_y  = numpy.concatenate((y_train, y_train2), axis=0)


# # fits the model on batches with real-time data augmentation:
history = model.fit_generator(datagen.flow(result_x, result_y, batch_size=35),
                    steps_per_epoch=len(X_train) / 32, epochs = 12)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Epoch 1/12
1050/1050 [==============================] - 23s 22ms/step - loss: 0.3065 - acc: 0.9040
Epoch 2/12
1050/1050 [==============================] - 21s 20ms/step - loss: 0.0952 - acc: 0.9748
Epoch 3/12
1050/1050 [==============================] - 23s 22ms/step - loss: 0.0708 - acc: 0.9809
Epoch 4/12
1050/1050 [==============================] - 23s 22ms/step - loss: 0.0638 - acc: 0.9834
Epoch 5/12
1050/1050 [==============================] - 27s 26ms/step - loss: 0.0572 - acc: 0.9851
Epoch 6/12
1050/1050 [==============================] - 22s 21ms/step - loss: 0.0567 - acc: 0.9866
Epoch 7/12
1050/1050 [==============================] - 22s 21ms/step - loss: 0.0483 - acc: 0.9874
Epoch 8/12
1050/1050 [==============================] - 22s 21ms/step - loss: 0.0395 - acc: 0.9899
Epoch 9/12
1050/1050 [==============================] - 22s 21ms/step - loss: 0.0398 - acc: 0.9899
Epoch 10/12
1050/1050 [==============================] - 21s 

In [17]:
# model.fit(X_train, y_train, epochs= 32 , batch_size=200, validation_split = 0.2)
scores = model.evaluate(X_test, y_test, verbose = 10 )
print ( scores )


[0.16060572609864276, 0.9895238095238095]


In [18]:
test_set = (test.values).reshape(-1, 28, 28 , 1).astype('float32')

res = model.predict(test_set)
res = numpy.argmax(res,axis = 1)
res = pd.Series(res, name="Label")
submission = pd.concat([pd.Series(range(1 ,28001) ,name = "ImageId"),   res],axis = 1)
submission.to_csv("cnn_mnist_datagen.csv",index=False)
submission.head(10)

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
5,6,7
6,7,0
7,8,3
8,9,0
9,10,3


Note that these results can be further optimized and regulized but i will leave that for you. Knowing how to optimize the results is very important and can be done using error analysis techniques.

### This tutorial is written by AbdElRhman ElMoghazy.

### Refrences ,Textbooks and Tutorials :
Hands on machine learning with scikit-learn and TensorFlow by Aurélien Géron

Pyhron machine learning 2nd edition by Sebastian Raschka ,Vahid Mirjalili

http://www.deeplearningbook.org/

https://keras.io/

https://machinelearningmastery.com/handwritten-digit-recognition-using-convolutional-neural-networks-python-keras/

https://codelabs.developers.google.com/codelabs/cloud-tensorflow-mnist/index.html?index=..%2F..%2Findex#0